# Creative Writing v3

Create a SequentialChain with 4 LLMChain(s)
Define the JSON Schema

1. LLMChain 1 : Generate 3 plan
2. LLMChain 2 : Vote the best plan out of the 3 plans
3. LLMChain 3 : Generate 3 writeups from the best plan
4. LLMChain 4 : Vote the best passage out of the 3 passages
5. SequentialChain : Combine the 4 chains above

To keep things simple you can ues the same LLM for all chains

### Solution
#### Part-1 
Setup the JSON classes & ouuput parsers

#### Part-2
Create the sequential chaain that retuurns a JSON wrapper with all outputs from LLM


### Unit Testing : Step by ste execution

Helpful to debug in case you have issues with ouput format issues.
n
* In order to test the working of each step
* UNCOMMENT the code in cells with the heading OPTIONAL
* Run each step one-by-one and go through the output

### NOTE
* Code & prompts are intentionally kept simple to make it easy to understand
* Code was tested with OpenAI GPT
* Prompts will need to be adjusted for other models
* Changes in model parameters may result in different output
* Format issues:
  - You may see a parsing failure
  - You can reduce the chance of error by providing 2 or 3 examples in each template
 
### Challenge problem
* Make a copy of this notebook
* Use a different model
    - Adjust prompts as neede..

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

* The API key file file will not be available
* You will be prompted to provide the API Token

Uncomment & run the code in the cell below:

In [1]:
## The script is downloaded and run to setup the utils folder

# !curl -H "Accept: application/vnd.github.VERSION.raw" https://raw.githubusercontent.com/acloudfan/gen-ai-app-dev/main/Setup/gcsetup.sh  > gcsetup.sh
# !chmod u+x gcsetup.sh
# !./gcsetup.sh -l

## Setup environment

In [2]:
from dotenv import load_dotenv
import os
from IPython.display import Markdown, JSON

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

## Create LLM

In [3]:
import sys
 
# setting path
sys.path.append('../')

from utils.create_llm import create_gpt_llm, create_cohere_llm, create_ai21_llm, create_ollama_llm

# OpenAI GPT args
openai_args = {"max_tokens": -1, "temperature": 0.5}
llm = create_gpt_llm(openai_args)

llm = create_gpt_llm()

## Use Ollama
# llm = create_ollama_llm()

## Writeup instructions

In [4]:
writeup_instructions = """
Write a passage of under 100 words to describe our new product 'Javelin 20 High Performance Shoes'.
The title must contain the sentence 'A new you'.The first sentence must contain a few of these words 'motivation, workout, sports, attractive'.
The last few sentences must motivate the reader to visit out website http://acme.com/jav20. The passage must contain the phrase 'trendsetter'.
"""

## Part-1

### Prompts & Ouput Parsers

https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html#langchain_core.output_parsers.json.JsonOutputParser

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

### 1. Template, JSON Schema & Parser : Generate 3 plans

In [6]:
template_1 = """
you are an expert in writing creative marketing material.

Generate a list of writing plans for executing the task with the following instructions.
The size of the plans list will be three. Keep the number of steps for each plan between 4 and 10.


{task_instructions}

Format Instructructions:
{format_instructions}

""" 

# 1. Define the JSON classes for plans & list
class PassagePlans(BaseModel):
    plan_number: int = Field(description = "this is the plan number")
    plan_steps: list[str] = Field(description = "these are the plan steps")

class ListPassagePlansOutput(BaseModel):
    plans: list[PassagePlans] = Field(description = "list of PassagePlans")

# Create the parser
output_parser_generate_plan = JsonOutputParser(pydantic_object=ListPassagePlansOutput)

### 2. Template & JSON Schema: Vote the best plan

In [7]:
template_2 = """
you are an expert in writing creative marketing material.

three experts created separate writing plans for carrying out the following task.
your job is to review the three plans, identify the best plan out of the three and provide three reasons for selecting the plan.
you must strictly follow the format instructions to generate your output.

Task Instructions:
{task_instructions}

Plans:
{proposed_plans}

Format Instructions:
{format_instructions}

""" 

# Create the class that represents output of step 2 i.e., best voted plan
class VoteBestPlan(BaseModel):
    plan_number: int = Field(description = "this is the plan number for the plan voted as best")
    plan_steps: list[str] = Field(description = "these are the plan steps")
    reasons: list[str] = Field(description = "provides 3 reasons why the plan was voted the best")

# Create the parser
best_plan_parser = JsonOutputParser(pydantic_object=VoteBestPlan)

### 3. Template & JSON Schema : Generate 3 plans with the best voted plan

In [8]:
template_3 = """
You are a writer for marketing material.
Create three passages using the following task instructions and the provided plan.
You must strictly follow the format instructions for the generated passages.

Task Instructions:
{task_instructions}

Plan:
{best_voted_plan_details}

Format Instructions:
{format_instructions}

"""

# Output for generating 3 passages from the best plan
class ThreePassgesWithBestPlan(BaseModel):
    passages: list[str] = Field(description = "list containing the generated passages")

# Create the parser
generate_passages_parser = JsonOutputParser(pydantic_object=ThreePassgesWithBestPlan)

### 4. Template & JSON Schema : Select the best passage

In [9]:
template_4 = """
you are an expert in writing creative marketing material.

three experts were given a writing assignment with task instructions.
your task is to review the 3 passages and pick the best passage out of the three. 
provide three reasons why you selected the passage.
you must strictly follow the format instructions for the generated passages.

Task Instructions:
{task_instructions}

Passages:
{generated_passages_with_best_plan}

Your output will be in the following format.

Format Instructions:
{format_instructions}

"""

# Define schema for output from last step

class BestPassage(BaseModel):
    passage_number: int = Field(description = "this is the passage number for the best passage out of the three passages")
    reasons: list[str] = Field(description = "three reasons in a list on why you selected the passage")

# Create the parser
best_passage_parser = JsonOutputParser(pydantic_object=BestPassage)

## Part-2

Setup the chains

### Utility function

Creates the chain with given set of parameters

In [10]:
from langchain import PromptTemplate, LLMChain

def  create_chain(template, llm, input_variables=[], partial_variables={},  output_parser=None, output_key='text'):
    """
        Utility function to create the LLMChain
    """
    # create the prompt template
    prompt_template = PromptTemplate(
        template = template,
        input_variables = input_variables,
        partial_variables = partial_variables
    )

    # create the chain
    if output_parser:
        # Legacy
        # llm_chain  = LLMChain(
        #     prompt = prompt_template,
        #     llm = llm,
        #     output_key = output_key,
        #     output_parser = output_parser
        # )

        # Use LCEL
        llm_chain = prompt_template | llm | output_parser
    else:
        # Legacy
        # llm_chain  = LLMChain(
        #     prompt = prompt_template,
        #     llm = llm,
        #     output_key = output_key,
        # )
        llm_chain = prompt_template | llm

    return prompt_template, llm_chain

### 1. Generate the plans

#### Setup prompt & create LLMChain

In [11]:
partial_variables = {"format_instructions": output_parser_generate_plan.get_format_instructions(), "task_instructions": writeup_instructions}
prompt_template, llm_chain_generate_plan = create_chain(template_1, 
                                       llm, 
                                       partial_variables=partial_variables,
                                       output_key="proposed_plans",
                                       output_parser=None  
                        )



#### Invoke Chain  - OPTIONAL STEP for unit testing

In [12]:
# print(prompt_template.format())
# response = llm_chain_generate_plan.invoke({})


you are an expert in writing creative marketing material.

Generate a list of writing plans for executing the task with the following instructions.
The size of the plans list will be three. Keep the number of steps for each plan between 4 and 10.



Write a passage of under 100 words to describe our new product 'Javelin 20 High Performance Shoes'.
The title must contain the sentence 'A new you'.The first sentence must contain a few of these words 'motivation, workout, sports, attractive'.
The last few sentences must motivate the reader to visit out website http://acme.com/jav20. The passage must contain the phrase 'trendsetter'.


Format Instructructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of th

#### Parse the response - OPTIONAL STEP for unit testing

In [17]:

# proposed_plans = output_parser_generate_plan.parse(response['proposed_plans'])

# JSON(proposed_plans)

JSONDecodeError: Unterminated string starting at: line 25 column 9 (char 1035)

### 2. Vote the best plan

#### Setup prompt template & LLMChain

In [13]:
partial_variables = {"format_instructions": best_plan_parser.get_format_instructions(), "task_instructions": writeup_instructions}
prompt_template, llm_chain_vote_plan = create_chain(template_2, 
                                       llm, 
                                       input_variables=['proposed_plans'],
                                       partial_variables=partial_variables,
                                       output_key="best_voted_plan_details"
                        )



#### Invoke chain - OPTIONAL STEP for unit testing

In [14]:
# print(prompt_template.format(proposed_plans=proposed_plans['plans']))
# response = llm_chain_vote_plan.invoke({'proposed_plans': proposed_plans})

#### Parse response - OPTIONAL STEP for unit testing

In [15]:
# best_voted_plan_details = best_plan_parser.parse(response['best_voted_plan_details'])
# JSON(best_voted_plan_details)

<IPython.core.display.JSON object>

### 3. Generate 3 passages with best plan

#### Setup prompt and LLMChain

In [16]:


partial_variables = {"format_instructions": generate_passages_parser.get_format_instructions(), "task_instructions": writeup_instructions}
prompt_template, llm_chain_generate_passages_on_best_plan = create_chain(template_3, 
                                       llm, 
                                       input_variables=['best_voted_plan_details'],
                                       partial_variables=partial_variables,
                                       output_key="generated_passages_with_best_plan"
                        )



#### Invoke chain - OPTIONAL STEP for unit testing

In [17]:
# response = llm_chain_generate_passages_on_best_plan.invoke({'best_voted_plan_details': best_voted_plan_details})

#### Parse - OPTIONAL STEP for unit testing

In [18]:
# three_passages = generate_passages_parser.parse(response['generated_passages_with_best_plan'])
# JSON(three_passages)

<IPython.core.display.JSON object>

### 4 Vote the best passage

#### Setup prompt & LLMChain

In [19]:
partial_variables = {"format_instructions": best_passage_parser.get_format_instructions(), "task_instructions": writeup_instructions}
prompt_template, llm_chain_vote_best_passage = create_chain(template_4, 
                                       llm, 
                                       input_variables=['generated_passages_with_best_plan'],
                                       partial_variables=partial_variables,
                                       output_key="best_passage"
                        )

#### Invoke chain - OPTIONAL STEP for unit testing

In [20]:
passages = three_passages['passages']

response  = llm_chain_vote_best_passage.invoke({'generated_passages_with_best_plan': passages})

#### Parse - OPTIONAL STEP for unit testing

In [21]:
best_passage_info = best_passage_parser.parse(response['best_passage'])

JSON(best_passage_info)

<IPython.core.display.JSON object>

### 5. Create SequentialChain

In [22]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains=[llm_chain_generate_plan, llm_chain_vote_plan, llm_chain_generate_passages_on_best_plan, llm_chain_vote_best_passage ],
    input_variables = [],
    output_variables = ["proposed_plans", "best_voted_plan_details", "generated_passages_with_best_plan", "best_passage"],
    verbose = True,
)



In [23]:
response = sequential_chain.invoke({}, verbose=True)



> Entering new SequentialChain chain...

> Finished chain.


In [24]:
JSON(response)

<IPython.core.display.JSON object>

In [25]:
proposed_plans = output_parser_generate_plan.parse(response['proposed_plans'])
best_voted_plan_details = best_plan_parser.parse(response['best_voted_plan_details'])
generated_passages_with_best_plan = generate_passages_parser.parse(response['generated_passages_with_best_plan'])
best_passage = best_passage_parser.parse(response['best_passage'])

In [26]:
wrapper_for_ui = {
    "proposed_plans" : proposed_plans,
    "best_voted_plan_details": best_voted_plan_details,
    "generated_passages_with_best_plan": generated_passages_with_best_plan,
    "best_passage": best_passage
}

JSON(wrapper_for_ui)

<IPython.core.display.JSON object>